## TableGan 생성을 하기 위한 데이터 만들기
- feature drop
- Specific feature datatype 변환
- NaN value drop

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import os

import torch

import warnings
warnings.filterwarnings("ignore") 

In [2]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')
print(device)

cuda


In [3]:
data_path = "Dataset/loan_data.csv"
data = pd.read_csv(data_path, low_memory=False)

### Feature Drop 과정

In [4]:
#feature drop
drop_features = ['Unnamed: 0','purpose', 'title','id', 'pymnt_plan', 'policy_code','url']
drop_data_leakage = ['total_pymnt','total_pymnt_inv','out_prncp',
                     'out_prncp_inv','recoveries','collection_recovery_fee',
                     'last_pymnt_d','last_pymnt_amnt','issue_d',
                     'next_pymnt_d','last_credit_pull_d','total_rec_prncp', 
                    'total_rec_int','total_rec_late_fee','last_fico_range_high',
                    'last_fico_range_low'
                     ]
data = data.drop(columns=drop_features + drop_data_leakage)

In [5]:
data_x = data.drop(columns='loan_status')
data_y = data[['loan_status']]

print(len(data_x.columns))

118


In [6]:
#data 타입 변환
#퍼센트를 float형태로 바꾸기
def converting_cat_to_num(x):
    if not isinstance(x,str):
        return x  
    return float(x.split("%")[0].lstrip())

#zip code 뒤에 x 2개 빼고 float형 변환
def filter_zip_code(x):
    if not isinstance(x,str):
        return x
    return int(x[0:-2])

def to_data_tranfromer(x):
    if not isinstance(x, str):
        return np.nan
    date = x.split('-')
    if len(date) != 2:
        return np.nan  # 예상치 못한 포맷 처리
    month, year = date[0], date[1]
    return int(year + enum.get(month, "00"))  # "YYYYMM" 형식으로 변환

## 시간 데이터 확인
to_data_features = ["earliest_cr_line",  \
                    "sec_app_earliest_cr_line", "hardship_start_date",\
                    "hardship_end_date", "payment_plan_start_date"]

enum = {'Jan':"01", 'Feb':"02", 'Mar':"03", 
        'Apr':"04", 'May':"05", 'Jun':"06",
        'Jul':"07", 'Aug':"08", 
        'Sep':"09", 'Oct':"10", 
        'Nov':"11", 'Dec':"12"}

# Converting numerically data stored as categorical
data_x['int_rate'] = data_x['int_rate'].map(converting_cat_to_num)
data_x['revol_util'] = data_x['revol_util'].map(converting_cat_to_num)

# Filtering the zip code
data_x["zip_code"] = data_x["zip_code"].map(filter_zip_code)


for name in to_data_features:
    data_x[name] = data_x[name].map(to_data_tranfromer)

print("Successfully converted!")

Successfully converted!


In [7]:
# Analyzing the numerical data generally
number_of_missing_values = {}
percentage_of_missing_values = {}
number_of_duplicate_elements = {}
ratio_of_duplicates = {}

# Check columns
columns_name_to_check = data_x.columns
#columns_name_to_check = columns_name_to_check.drop("loan_status")
# Calculating
for name in columns_name_to_check:
    number_of_nulls = data_x[name].isnull().sum()
    number_of_missing_values[name] = number_of_nulls
    percentage_of_missing_values[name] = (number_of_nulls/len(data_x[name]))*100
    
for name in columns_name_to_check:
    number_of_duplicate_elements[name] = (data_x[name].count() - data_x[name].nunique())

for name in columns_name_to_check:
    ratio_of_duplicates[name] = number_of_duplicate_elements[name]/data_x[name].count()    

# Restructuring the Gathered Information
reassemble_data = {
                  "Attribute Names" : list(ratio_of_duplicates.keys()),
                  "Number of Missing Values" : list(number_of_missing_values.values()),
                  "Missing Values %" : list(percentage_of_missing_values.values()),
                  "Number of Duplicates" : list(number_of_duplicate_elements.values()),
                  "Ratio of Duplicates" : list(ratio_of_duplicates.values())
                  }

statistics_about_data =  pd.DataFrame(reassemble_data)

# Visualizing the Data
statistics_sorted = statistics_about_data.sort_values(by = 'Missing Values %') 
statistics_sorted.style.set_properties(subset=['Attribute Names'], **{'font-weight': 'bold'})\
.background_gradient(axis=0) #subset='Number of Duplicates')

,Attribute Names,Number of Missing Values,Missing Values %,Number of Duplicates,Ratio of Duplicates
0,loan_amnt,1,0.000034,2923920,0.999463
31,application_type,1,0.000034,2925490,0.999999
28,initial_list_status,1,0.000034,2925490,0.999999
25,revol_bal,1,0.000034,2812158,0.961260
19,fico_range_high,1,0.000034,2925444,0.999984
18,fico_range_low,1,0.000034,2925444,0.999984
14,addr_state,1,0.000034,2925441,0.999983
12,verification_status,1,0.000034,2925489,0.999999
10,home_ownership,1,0.000034,2925486,0.999998
117,debt_settlement_flag,1,0.000034,2925490,0.999999


In [8]:
#Missing value ratio가 5를 넘어가는 것들은 모두 drop
features_to_remove = list(statistics_sorted.loc[(statistics_sorted["Missing Values %"] > 5)]["Attribute Names"])
print(f"List of features that need to be removed: {len(features_to_remove)}")

for  name in features_to_remove:
    data_x.drop(labels=name, axis=1, inplace=True)

print("Non-high presence features have been deleted successfully!")

List of features that need to be removed: 53
Non-high presence features have been deleted successfully!


In [9]:
print(f"The number of features : {len(data_x.columns)}\nThe target feature : {data_y.columns}")

The number of features : 65
The target feature : Index(['loan_status'], dtype='object')


### NaN value drop

In [10]:
print(f"Before drop: {data_x.shape[0]} samples")
data_t = pd.concat([data_x, data_y], axis = 1)
data_x_cleaned = data_t.dropna()

data_x_cleaned.info()

Before drop: 2925493 samples
<class 'pandas.core.frame.DataFrame'>
Index: 2779991 entries, 42536 to 2925492
Data columns (total 66 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   loan_amnt                   float64
 1   funded_amnt                 float64
 2   funded_amnt_inv             float64
 3   term                        object 
 4   int_rate                    float64
 5   installment                 float64
 6   grade                       object 
 7   sub_grade                   object 
 8   home_ownership              object 
 9   annual_inc                  float64
 10  verification_status         object 
 11  zip_code                    float64
 12  addr_state                  object 
 13  dti                         float64
 14  delinq_2yrs                 float64
 15  earliest_cr_line            float64
 16  fico_range_low              float64
 17  fico_range_high             float64
 18  inq_last_6mths              float6

### Dataset Class binary로 변환

In [11]:
data_x_cleaned['loan_status'].value_counts()

loan_status
Fully Paid            1415301
Current                996786
Charged Off            345458
Late (31-120 days)      10915
In Grace Period          7114
Issued                   2040
Late (16-30 days)        2007
Default                   370
Name: count, dtype: int64

In [12]:
#loan_data 특정 class만 살리고 Charged off -> Defalut class로 바꾸기기
data_x_cleaned = data_x_cleaned[data_x_cleaned['loan_status'].isin(["Fully Paid", "Charged Off", "Default"])]
print(data_x_cleaned['loan_status'].value_counts())

data_x_cleaned['loan_status'] = data_x_cleaned['loan_status'].replace("Charged Off", "Default")
print('\n',data_x_cleaned['loan_status'].value_counts())

loan_status
Fully Paid     1415301
Charged Off     345458
Default            370
Name: count, dtype: int64

 loan_status
Fully Paid    1415301
Default        345828
Name: count, dtype: int64


### data_x_cleaned 데이터 특징 
- total sample : 2779991
    - Fully - Paid : 1415301
    - Default : 345828
- Feature : 66 columns

In [13]:
### data_x_cleaned에서 30만개 가량 loan-status가 Default인 것만 csv파일 만들기
data_defalut = data_x_cleaned[data_x_cleaned['loan_status'] == 'Default']

data_defalut['loan_status'].value_counts()

loan_status
Default    345828
Name: count, dtype: int64

In [ ]:
"""## csv파일로 만들기
address = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/"
data_defalut.to_csv(path_or_buf=address+"defalut_original.csv", index=False)"""